# Display tables resulting from an astroquery request

This example implements the addition of two tables :
- one resulting from an astroquery request on the [Simbad](https://simbad.cds.unistra.fr/simbad/ "https://simbad.cds.unistra.fr/simbad/") database
- the other is a table created locally

In [ ]:
from ipyaladin import Aladin
from astropy.table import QTable
import astropy.units as u
from astroquery.simbad import Simbad

## From a query

In [ ]:
table = Simbad.query_region("Messier 1", radius=0.03 * u.deg)
table

In [ ]:
aladin = Aladin(fov=0.4, target="Messier 1")
aladin

In [ ]:
aladin.add_table(table, shape="rhomb", color="lightskyblue", source_size=20)
# This line also works with camelCase instead of snake_case: sourceSize=20

## Display Astropy table created from scratch

In [ ]:
ra = [83.63451584700, 83.61368056017, 83.58780251600]
dec = [22.05652591227, 21.97517807639, 21.99277764451]
name = [
    "Gaia EDR3 3403818589184411648",
    "Gaia EDR3 3403817661471500416",
    "Gaia EDR3 3403817936349408000",
]
parallax = [1.7703, 0.5112, 0.3735] * u.mas

t = QTable(
    [ra, dec, name, parallax],
    names=("ra", "dec", "name", "parallax"),
    meta={"name": "my sample table"},
)

In [ ]:
aladin.add_table(t)

## Display the table with the error approximations
First, let's re-use the table from the Simbad query and specify the error columns and units, and the ellipse enclosed probability.
### Note
Ipyaladin only support oriented ellipse and radial errors for the moment.
Radial error can be specified with the `r` column in the table.

In [ ]:
error_dict = {
    "smaj": {"col": "coo_err_maj", "unit": u.mas},
    "smin": {"col": "coo_err_min", "unit": u.mas},
    "pa": {"col": "coo_err_angle", "unit": u.deg},
    # Let's the default value for the ellipse enclosed probability
    "ellipse_enclosed_probability": 0.39347,
}

And then add the table to the Aladin widget

In [ ]:
aladin2 = Aladin(target="M1", fov=0.2)
aladin2

In [ ]:
aladin2.add_table(table, error_dict=error_dict)